# Verification of the diagnosis by the RIBEIRO PRE-TRAINED Model 
  
- On this notebook we wiil look further into the stats of the RIBEIRO NN and confirm the results annonced by his team.  
- We will work over a part of the giant dataset of more thant 2 millions 12-lead ECG.  
- For that we take 30000 ECG that will construct our validation dataset. It represents aproximately 1.3% of the whole training dataset
- We will create 7 datasets, one for each cardiac disease and the last one if the patient is healthy.  
- Also, in order to understand why the prediction is not correct, a {disease}_prob dataset will be created and some of the ECG that causes problem to the machine will be printed.


# I. Determine diseases datasets  
  
First we analyse the 1.5%_dataset to find what are the index of the abnormal-ECG, we categorise them into 7 lists.  
Those lists will be limited to 2 thousands index, it means that our stats won't be perfect but they will approximate the real ones.


  ### I.1. Open datas and take only 30000 samples


Taking the annotated ECG in order to compare the prediction to the annotation at the end.  
We want 1 or 2 thousands ECG of each diseases, then 30000 datas looks alright to do so.

In [4]:
import numpy as np
import h5py
from tqdm import tqdm

TEST_SHAPE = 30000 
PATH_TO_GIANT = '/media/cotxetjordi/TOSHIBA EXT/Data_2/'

with h5py.File(PATH_TO_GIANT + "preprocessed_traces15pc.hdf5", "r") as f:
    i = 0
    signals = []
    id_exams = []
    ids = list(f["id_exam"][:60000])  #Only 40000 raw data needed

    for index,data in tqdm(enumerate(f["signal"])):

        id = ids[index]

        if id < 2200000: # Only takes exams with annotations available in annotations.csv
            
            signals.append(data)
            id_exams.append(id)
            i+=1
            
        if i == TEST_SHAPE:
            break
            
        
signals = np.array(signals)
id_exam = np.array(id_exams)
print("La taille du dataset est de :", np.shape(signals))
print(len(id_exams))


40635it [00:07, 5340.05it/s]


La taille du dataset est de : (30000, 4096, 12)
30000


### I.2.a) Save the datas to use it later 


In [6]:
with h5py.File(PATH_TO_GIANT + 'datas_to_verif.hdf5', 'a') as new_file:
    id_exam_data = new_file.create_dataset(name = "id_exam",data = id_exams)
    signals_data = new_file.create_dataset(name = "signal",data= = signals)
    

### I.2.b) Load them

In [40]:
import h5py
PATH_TO_GIANT = '/media/cotxetjordi/TOSHIBA EXT/Data_2/'

with h5py.File(PATH_TO_GIANT + 'datas_to_verif.hdf5', 'r') as new_file:
    id_exams = new_file["id_exam"]
    X = new_file["signal"][()]



(30000, 4096, 12)


## I.3. Create 7 datasets, one for each disease
### 3.1. Find the annotation of each signal of our dataset and divide it into 7 datasets 



In [43]:
import time
import csv
PATH_TO = '/media/cotxetjordi/TOSHIBA EXT/Data_2/' #Path to your csv file 
counter = -1
start_time = time.time()
with open(PATH_TO + 'annotations_clean.csv','r', newline='') as csvfile:
        COUNTER = 0
        i = 1
        carac = {"np":[],"1dAVB":[],"RBBB":[],"LBBB":[],"SB":[],"AF":[],"ST":[]}
        f = csv.reader(csvfile)
        indexes = list(carac.keys())
        
        
        for line in f:

            if i in id_exams:
                COUNTER +=1
                if "1" in line and line.count("1") == 1 : #Just taking the "mono-disease" ECG to analyse
                    disease_ids = carac[indexes[line.index("1")+1]]
                    if len(disease_ids) < 2000:
                        disease_ids.append(id_exams.index(i))

                elif len(carac["np"]) < 2000:
                    carac["np"].append(id_exams.index(i))
                    
            if i % 50000 == 0:
                    print(COUNTER)
                    print("--- %s seconds ---" % (time.time() - start_time))            
            if COUNTER == 12000:
                break
                
            i += 1

           
print("--- %s seconds ---" % (time.time() - start_time)) 


962
--- 159.48490262031555 seconds ---
1864
--- 317.4650228023529 seconds ---
2693
--- 475.2715952396393 seconds ---
3552
--- 631.7928097248077 seconds ---
4381
--- 787.9370865821838 seconds ---
5318
--- 943.591845035553 seconds ---
6253
--- 1099.3485352993011 seconds ---
7151
--- 1254.929848432541 seconds ---
8021
--- 1410.2463309764862 seconds ---
8893
--- 1564.9671349525452 seconds ---
9839
--- 1718.799234867096 seconds ---
10720
--- 1873.2899703979492 seconds ---
11640
--- 2027.871402978897 seconds ---
--- 2086.9377748966217 seconds ---


### I.3.a) Save the datas

In [44]:
import numpy as np

for index, (dis,ids) in enumerate(carac.items()) :

    np.save(dis,ids)



np
1dAVB
RBBB
LBBB
SB
AF
ST


### I.3.b) Load the 7 datasets

In [45]:
import numpy as np
carac = {"np":[],"1dAVB":[],"RBBB":[],"LBBB":[],"SB":[],"AF":[],"ST":[]}
for index,(key,ids) in enumerate(carac.items()):
    
    carac[key] = np.load(key+ ".npy")

    print(key,"has", len(carac[key]),"samples")
    



(2000,)
np has 2000 samples
(135,)
1dAVB has 135 samples
(272,)
RBBB has 272 samples
(140,)
LBBB has 140 samples
(164,)
SB has 164 samples
(189,)
AF has 189 samples
(282,)
ST has 282 samples


### I.4 Create an array composed by 7 arrays of disease-acquisition

In [47]:
X_test_all = np.array([[X[index] for index in ids] for ids in carac.values()]) #don't forget to converte each of the 7 lists when you will predict using the pre-trained model

    

(2000, 4096, 12)


<ipython-input-47-fcfa0ef8bd8e>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_all = np.array([[X[index] for index in ids] for ids in carac.values()]) #don't forget to converte each of the 7 lists when you will predict using the pre-trained model


## II. Prediction test
### II.1 Load and compile the model

In [64]:
cd Documents/Stage_Notebook/Rendu/

/home/cotxetjordi/Documents/Stage_Notebook/Rendu


In [89]:

# %% Import packages
import tensorflow.compat.v1 as tf  #Fait appel à la binliothèque Tensorflow Version 1.xx
tf.disable_v2_behavior()
import numpy as np
import warnings
import argparse

warnings.filterwarnings("ignore")
from keras.models import load_model
from keras.optimizers import Adam
import h5py

PATH_TO_MODEL  = 'model.hdf5'



# Import model
model = keras.models.load_model(PATH_TO_MODEL,compile=False)

#model = load_model(PATH_TO_MODEL, compile=False)
model.compile(loss='binary_crossentropy', optimizer=Adam())


print("Model saved")

Model saved


### II.2. Predict

In [88]:
predictions = []
prediction = model.predict(np.array([X_test_all[2][1]]))
for X in X_test_all:
    prediction = model.predict(np.array([X[0]]),verbose = 1)
print(prediction)

[[3.1088882e-07 7.6678202e-09 7.9162641e-09 5.4571192e-10 3.7799751e-08
  4.1579469e-09]]
